La empresa desea saber si tiene los recursos necesarios para soportar la demanda de los usuarios e incrementar el numero de ticktes por día. En base a ello, diseñe y desarrolle un modelo y/o script que permita realizar la simulación del proceso de pasar cuenca-aire con las siguientes especificaciones.

• La empresa tiene 3 carriles de revisión vehicular.

• Un solo carril es habilitado para carros de gran tonelaje, busetas, bus, camiones, solo el 15 % de vehiculos que llegan son de este tipo.

• Cada 3 minutos llega un nuevo vehiculo a revision.

• Se sigue el siguiente proceso 1) Llega el vehiculo al centro, se estaciona (3-5 minutos), saca un turno (1-3 minutos) y pagan la matricula (5 - 10 minutos).

2) Si esta vacio pasa directo a la revisión que tiene una media de 20 minutos con una desviacion estandar de 10 minutos.

3) Solo el 60 % de vehiculos pasa sin ninguna notificación, el 40% salen realizan el arreglo (2 dias) y nuevamente realizan el proceso.

4) Despues de pasar la revisión, se les pega el sello (1 – 3 minutos) .

5) Finalmente, salen (5 – 10 minutos).

Generar un modelo de simulación por 30 dias y generar las siguientes graficas de resultados:

• Numero de vehiculos atendidos por dia, y por carril • Porcentaje total de vehiculos aprobados y no aprobados en cuenca aire por dia. • Tiempos del proceso de matriculación, por dia.

In [1]:
!pip install simpy

In [40]:
import simpy
import random
from scipy.stats import uniform
from random import expovariate
from scipy import stats
import matplotlib.pyplot as plt

repetir = []
tiempo={}

pasa = []
class PROCESO(object):
  def __init__(self, environment):
    self.env=environment
    self.entrada_liviano = simpy.Resource(environment, 2)
    self.entrada_pesado = simpy.Resource(environment, 1)
    self.turno = simpy.Resource(environment, 1)
    self.pago = simpy.Resource(environment, 1)
    self.revision = simpy.Resource(environment, 1)
    self.sello = simpy.Resource(environment, 1)
    self.salida = simpy.Resource(environment, 1)

  def ingreso(self, objeto, pesado):
    yield self.env.timeout(random.randint(3,5))
    #print(f'Estaciona {objeto} tiempo {env.now}')
    if pesado:
      self.entrada_pesado.release(objeto)
    else:
      self.entrada_liviano.release(objeto)

  def sacar_turno(self, objeto):
    yield self.env.timeout(random.randint(1,3))
    #print(f'Saca turno {objeto} tiempo {env.now}')
    self.turno.release(objeto)

  def pagar_matricula(self, objeto):
    yield self.env.timeout(random.randint(5,10))
    #print(f'Paga matricula {objeto} tiempo {env.now}')
    self.pago.release(objeto)

  def revisar(self, objeto):
    yield self.env.timeout(random.normalvariate(20, 10))#desviacion estandar 10
    self.revision.release(objeto)
    
    if random.randint(1,100) < 60:
      print(f'Pasa Revision {objeto} tiempo {env.now}')
    else:
      print(f'No pasa Revision {objeto} tiempo {env.now}')
      pasa.append(objeto)

  def sellar(self, objeto):
    yield self.env.timeout(random.randint(1,3))
    #print(f'Sellado {objeto} tiempo {env.now}')
    self.sello.release(objeto)
    

  def salir(self, objeto, pesado):
    yield self.env.timeout(random.randint(5,10))
    #print(f'Sale {objeto} tiempo {env.now}')
    self.salida.release(objeto)
    if objeto in pasa:
      print(env.now+3)
      repetir.append([objeto, env.now+3, pesado, True])#2 dias es 2880 minutos
      pasa.remove(objeto)
    


def llegada_objeto(env, nombre, proceso, pesado, repite):
  # Especificamos que vamos a usar un recurso (Resource) que representa
  if repite:
    with proceso.revision.request() as revision:
      yield revision
      yield env.process(proceso.revisar(nombre, pesado))
    
  else:
    if pesado:
      with proceso.entrada_pesado.request() as entrada:
        yield entrada
        yield env.process(proceso.ingreso(nombre, pesado))
    else:
      with proceso.entrada_liviano.request() as entrada:
        yield entrada
        yield env.process(proceso.ingreso(nombre, pesado))

    with proceso.turno.request() as turno:
      yield turno
      yield env.process(proceso.sacar_turno(nombre))

    with proceso.pago.request() as pago:
      yield pago
      yield env.process(proceso.pagar_matricula(nombre))

    with proceso.revision.request() as revision:
      yield revision
      yield env.process(proceso.revisar(nombre))
  
  if nombre not in pasa:
    with proceso.sello.request() as sello:
      yield sello
      yield env.process(proceso.sellar(nombre))

  with proceso.salida.request() as salida:
    yield salida
    yield env.process(proceso.salir(nombre, pesado))


def ejecutar_simulacion(env):
  global error 
  proceso = PROCESO(env)
  # Creamos las llegadas de los objetos iniciales
  i=0
  while True:
    # print(repetir)
    vehiculo = ''
    pesado = False
    if random.randint(1,100) < 85:
      vehiculo = 'Liviano-%d'%(i+1)
      pesado = False
    else:
      vehiculo = 'Pesado-%d'%(i+1)
      pesado = True
    
    if len(repetir) > 0 :
      for r in repetir:
        if int(r[1]) in range(env.now-10, env.now+10, 1):
          print(f'Reingresa {r[0]} en tiempo {env.now}')
          env.process(llegada_objeto(env, r[0], proceso, r[2], r[3]))
          repetir.remove(r)

    yield env.timeout(3)
    print(f'Llega {vehiculo} en tiempo {env.now}')
    env.process(llegada_objeto(env, vehiculo, proceso, pesado, False))
    i+=1
    # if i >= 10:
    #   break
print('Simular')  
random.seed(12)
env=simpy.Environment()
env.process(ejecutar_simulacion(env))
env.run(80)
# env.run(43200)


Simular
Llega Liviano-1 en tiempo 3
Llega Liviano-2 en tiempo 6
Llega Liviano-3 en tiempo 9
Llega Liviano-4 en tiempo 12
Llega Liviano-5 en tiempo 15
Llega Liviano-6 en tiempo 18
Llega Liviano-7 en tiempo 21
Llega Liviano-8 en tiempo 24
Llega Liviano-9 en tiempo 27
Llega Liviano-10 en tiempo 30
Llega Pesado-11 en tiempo 33
Llega Liviano-12 en tiempo 36
Llega Liviano-13 en tiempo 39
Llega Liviano-14 en tiempo 42
No pasa Revision Liviano-1 tiempo 43.09268621120768
Pasa Revision Liviano-2 tiempo 44.052469545766215
Llega Liviano-15 en tiempo 45
Llega Liviano-16 en tiempo 48
Llega Liviano-17 en tiempo 51
55.09268621120768
Llega Liviano-18 en tiempo 54
Reingresa Liviano-1 en tiempo 54
Llega Pesado-19 en tiempo 57
Llega Liviano-20 en tiempo 60
Pasa Revision Liviano-3 tiempo 61.115877226484145
Llega Pesado-21 en tiempo 63
Llega Liviano-22 en tiempo 66
Llega Liviano-23 en tiempo 69
Llega Liviano-24 en tiempo 72
Llega Liviano-25 en tiempo 75
Pasa Revision Liviano-4 tiempo 76.62012889626175
Llega